In [47]:
from langchain_google_genai import ChatGoogleGenerativeAI
google_api_key = ""

llm = ChatGoogleGenerativeAI(
    model= "gemini-2.5-flash",
    temperature=0.3,
    google_api_key=google_api_key,
)

In [48]:
from core.state import State

In [49]:
from langgraph.graph import StateGraph
import os
from logger import setup_logger
#workflow = StateGraph(State)

In [50]:
pip install langchain_tavily

In [51]:
from langgraph.prebuilt import create_react_agent
from langchain_tavily import TavilySearch
from tools.basetool import eda_data


In [52]:
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = ""

In [53]:
llm = ChatGoogleGenerativeAI(
    model= "gemini-2.5-flash",
    temperature=0.3,
    google_api_key=google_api_key,
)

In [54]:
def create_suggest_agent():
    """Create the hypothesis agent"""

    web_search = TavilySearch(max_results=5)
    base_tools = [eda_data, web_search]
       
    
    system_prompt = '''
    Là một chuyên gia phân tích dữ liệu uy tín, nhiệm vụ của bạn là dựa trên thông tin của dữ liệu, hãy cho tôi biết dữ liệu có các cột gì và có thể phân tích được gì từ dữ liệu đó.
    

    '''
  


    return create_react_agent(
        llm, 
        base_tools,
        prompt = system_prompt,
        name= "suggest_agent"
    )

suggest_agent = create_suggest_agent()


In [55]:
def create_aboutdata_agent():
    """Create the hypothesis agent"""

    base_tools = [eda_data]
       
    
    system_prompt = '''
    Bạn là một chuyên gia phân tích dữ liệu. Nhiệm vụ của bạn là trả lời các câu hỏi của người dùng về thông tin dữ liệu dựa trên tool eda_data. 
    Chỉ cung cấp thông tin liên quan đến dữ liệu, bao gồm số dòng, số cột, kiểu dữ liệu, giá trị thiếu, hoặc thống kê cơ bản.
    Trả lời ngắn gọn, chính xác, bằng text, không sinh mã Python hoặc thực hiện các tác vụ phức tạp.
    Nếu không có dữ liệu hoặc thông tin không khả dụng, trả lời: "Không có thông tin dữ liệu để trả lời câu hỏi này."
    
    '''

    return create_react_agent(
        llm, 
        base_tools,
        prompt = system_prompt,
        name = "aboutdata_agent"
    )

aboutdata_agent = create_aboutdata_agent()

In [56]:
def create_hypothesis_agent(llm):
    """Create the hypothesis agent"""

    web_search = TavilySearch(max_results=5)
    base_tools = [eda_data, web_search]
       
    
    system_prompt = '''
    Bạn là một chuyên gia phân tích dữ liệu. Nhiệm vụ của bạn là sử dụng thông tin từ tool eda_data và web_search để gợi ý các insight ẩn hoặc giả thuyết nghiên cứu đằng sau dữ liệu, tận dụng thống kê, học máy, học sâu và AI. Các insight phải chính xác, khả thi, sáng tạo và được hỗ trợ bởi tài liệu.

    Quy trình:
    1. Phân tích dữ liệu từ eda_data để xác định các mẫu hình tiềm năng.
    2. Xây dựng 3-5 giả thuyết/insight (ví dụ: mối quan hệ giữa biến, dự đoán xu hướng).
    3. Phác thảo các bước kiểm định đơn giản (không sinh mã, chỉ mô tả).
    4. Sử dụng web_search để xác minh tính khả thi và độc đáo, trích dẫn 1-2 tài liệu tham khảo cho mỗi insight.

    Trả lời có cấu trúc: 
    - Insight 1: [Mô tả]. Khả thi vì [lý do]. Độc đáo vì [lý do]. Tài liệu: [link/tên].
    - ...

    Nếu dữ liệu không đủ hoặc không tìm thấy tài liệu, trả lời: "Không đủ thông tin để gợi ý insight đáng tin cậy."
    '''

    return create_react_agent(
        llm, 
        base_tools,
        prompt = system_prompt,
        name = "hypothesis_agent"
    )

hypothesis_agent = create_hypothesis_agent(llm)

In [57]:
from pydantic import BaseModel, Field

class CodeResponse(BaseModel):
    """Schema để định dạng câu trả lời cuối cùng của AI."""
    message: str = Field(description="Một lời giải thích hoặc tin nhắn thân thiện, súc tích cho người dùng.")
    code: str = Field(
        description="Một đoạn mã Python hoàn chỉnh nếu cần thiết để trả lời yêu cầu. Trả về chuỗi rỗng nếu không có code.")
    iscode: bool = Field(description="Trả về True nếu có đoạn mã trong trường 'code', ngược lại trả về False.")
    type: str = Field(
        description="Xác định loại output: 'text' (chỉ văn bản), 'img' (nếu code tạo ra đồ thị/hình ảnh), hoặc 'table_html' (nếu code tạo ra bảng dạng HTML).")



def create_code_agent(power_llm):
    """Create the code agent"""

    web_search = TavilySearch(max_results=3)
    base_tools = [eda_data, web_search]

    system_prompt = """
Bạn là một AI chuyên gia phân tích dữ liệu và lập trình viên Python. Nhiệm vụ của bạn là:

    1. Phân tích yêu cầu người dùng dựa trên thông tin từ tool `eda_data` hoặc dữ liệu người dùng cung cấp.
    2. Sinh mã Python chất lượng cao, sử dụng các thư viện Pandas, Matplotlib, Seaborn, NumPy để:
       - Tạo đồ thị trực quan hóa nếu được yêu cầu (lưu ý: không dùng `plt.show()`, lưu hình ảnh dưới dạng Base64).
       - Tạo bảng HTML nếu yêu cầu trả về DataFrame (đặt tên biến cuối là `df`, sử dụng `df.to_html()`).
       - In kết quả cuối cùng bằng `print()` nếu yêu cầu in thông tin (đảm bảo kết quả in là dòng cuối cùng).
    3. Mã Python phải:
       - Hoàn chỉnh, chạy độc lập, bao gồm tất cả import cần thiết (import pandas as pd, matplotlib.pyplot as plt, seaborn as sns, numpy as np).
       - Sử dụng đường dẫn file với `r""` (lấy từ người dùng hoặc `eda_data`).
       - Đặt tiêu đề, nhãn trục (x, y) bằng tiếng Việt có dấu.
    4. Trả về kết quả dưới dạng JSON theo schema `CodeResponse` với:
       - `message`: Lời giải thích ngắn gọn, thân thiện về kết quả hoặc mã.
       - `code`: Đoạn mã Python (chuỗi rỗng nếu không cần mã).
       - `iscode`: True nếu có mã, False nếu không.
       - `type`: 'text', 'img', hoặc 'table_html' tùy theo yêu cầu.
    5. Nếu không thể sinh mã (do thiếu dữ liệu, file không tồn tại, hoặc yêu cầu không rõ), trả về:
       - `message`: Giải thích lỗi (ví dụ: "Không tìm thấy file dữ liệu" hoặc "Yêu cầu không rõ, vui lòng cung cấp thêm thông tin").
       - `code`: Chuỗi rỗng.
       - `iscode`: False.
       - `type`: "text".

    Lưu ý:
    - Sử dụng tool `web_search` nếu cần thông tin bổ sung ngoài dữ liệu từ `eda_data`.
    - Đảm bảo mã tương tác đúng với file dữ liệu được cung cấp (đường dẫn file từ người dùng hoặc `eda_data`).
    - Không sinh mã cho các tác vụ không liên quan đến dữ liệu hoặc yêu cầu không rõ.
    """
    
    return create_react_agent(
        llm,
        base_tools,
        prompt = system_prompt,
        name = "code_agent",
        response_format = CodeResponse

    )
code_agent = create_code_agent(llm)

In [58]:
pip install langgraph_supervisor

In [59]:
from langgraph_supervisor import create_supervisor

def create_process_agent(power_llm):
    """Create the process/supervisor agent"""

    system_prompt = """
Bạn là một giám sát viên phân tích dữ liệu, chịu trách nhiệm điều phối các agent để hỗ trợ người dùng. Nhiệm vụ của bạn:

    1. **Phân loại yêu cầu**:
       - Thông tin dữ liệu (số dòng, cột, kiểu dữ liệu): Gọi `aboutdata_agent`.
       - Thực thi mã Python (tiền xử lý, vẽ biểu đồ, tính toán): Gọi `code_agent`.
       - Gợi ý insight/giả thuyết nghiên cứu: Gọi `hypothesis_agent`.

    2. **Xử lý yêu cầu đơn giản**:
       - Nếu yêu cầu chỉ cần gọi một agent, trả về kết quả ngay theo schema `CodeResponse` hoặc `InsightResponse`.

    3. **Xử lý yêu cầu phức tạp** (phân tích toàn diện):
       - Lập kế hoạch: Xác định mục tiêu và kết quả mong đợi.
       - Phân công: Gọi các agent phù hợp (`aboutdata_agent` cho thông tin, `code_agent` cho mã, `hypothesis_agent` cho insight).
       - Tích hợp: Kết hợp kết quả, đảm bảo tính nhất quán và logic.
       - Báo cáo: Tạo báo cáo ngắn gọn (dưới 500 từ) với insight, mã (nếu có), hình ảnh (nếu có), và tài liệu tham khảo.
       - Xác thực: Tinh chỉnh giả thuyết từ `hypothesis_agent` để đảm bảo rõ ràng, cụ thể, kiểm chứng được.

    4. **Xử lý lỗi**:
       - Nếu yêu cầu không rõ hoặc dữ liệu không đủ, trả về JSON với `message` giải thích, `code=""`, `iscode=False`, `type="text"`.
       - Nếu agent con trả về kết quả không phù hợp, yêu cầu lại hoặc trả về thông báo lỗi.

    5. **Đầu ra**:
       - Chỉ trả "HOÀN THÀNH" khi báo cáo đầy đủ, mã tái tạo được, hình ảnh rõ ràng, và tài liệu tham khảo chính xác.

    Lưu ý:
    - Ưu tiên trả lời nhanh cho yêu cầu đơn giản.
    - Đảm bảo mã Python (nếu có) tái tạo được, hình ảnh có nhãn tiếng Việt, và tài liệu tham khảo được trích dẫn đúng.
    """
    
    agents = [aboutdata_agent, code_agent, hypothesis_agent]
    return create_supervisor(
        model = power_llm,
        agents = agents,
        prompt = system_prompt,
        add_handoff_messages=True,
        add_handoff_back_messages=True,
        output_mode="last_message",
        response_format = CodeResponse,
        include_agent_name="inline"

    )

process_agent = create_process_agent(llm).compile()

In [60]:
from IPython.display import display, Image

display(Image(process_agent.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [ ]:
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [ ]:
for chunk in process_agent.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "thông tin và ý nghĩa các cột có trong dữ liệu, đường dẫn đến dữ liệu là: synthetic_ecommerce_data.csv",
            }
        ]
    },
):
    pretty_print_messages(chunk, last_message=True)

final_message_history = chunk["supervisor"]["messages"]

Update from node supervisor:


================================= Tool Message =================================
Name: transfer_to_aboutdata_agent

Successfully transferred to aboutdata_agent


Update from node aboutdata_agent:


================================= Tool Message =================================
Name: transfer_back_to_supervisor

Successfully transferred back to supervisor


Update from node supervisor:


================================== Ai Message ==================================
Name: supervisor

Để phân tích dữ liệu từ file `synthetic_ecommerce_data.csv`, tôi cần bạn cung cấp thêm thông tin về nội dung của file này. Cụ thể, bạn có thể cho tôi biết:

1.  **Tên các cột (column names)** có trong file.
2.  **Kiểu dữ liệu (data types)** của từng cột (ví dụ: số nguyên, số thực, chuỗi, ngày tháng).
3.  **Một vài dòng dữ liệu mẫu** để tôi có thể hình dung rõ hơn về định dạng và giá trị của dữ liệu.

Nếu bạn có thể cung cấp những thông tin này, tôi sẽ có thể giúp bạn hiểu rõ